In [1]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
df = pd.read_csv(path + 'cleaned_strings_df.csv')

In [2]:
df

,Unnamed: 0,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanzas,stanza_number,is_chorus
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She got a broke down El Camino, in the front y...",0,False
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,1,True
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She does her makeup and hair, to cook fried ch...",2,False
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,3,True
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"My hoosier girl is so fine, shake the watermel...",4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966418,161659,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,The thin gruel for millions of people,0,False
966419,161659,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,Nothing is left but a mountain of shoes Dig y...,1,False
966420,161659,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,The thin gruel for millions of people,2,True
966421,161659,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,Sometimes you live And sometimes you die But ...,3,False


In [10]:
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor

# Load the BART large MNLI model for zero-shot classification
classifier = pipeline(
    "zero-shot-classification",
    model="albert-base-v2"
)


def classify_emotion(text):
    emotion_labels = [
        "anger", "anticipation", "joy", "trust",
        "fear", "surprise", "sadness", "disgust"
    ]
    result = classifier(text, candidate_labels= emotion_labels)
    return result['labels'][0]  # Return the top label

def classify_emotions_in_parallel(texts, max_workers=4):
    # Using ThreadPoolExecutor to process texts in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(classify_emotion, texts))
    return results

# with Pool(processes= 4) as pool:
#     print(pool.map)

classify_emotion(df.loc[0, 'stanzas'])

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


'sadness'

In [11]:
classify_emotion(df.loc[1, 'stanzas'])

'fear'

In [14]:
classify_emotions_in_parallel(df['stanzas'].head(n= 1000))

['sadness',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'surprise',
 'joy',
 'joy',
 'fear',
 'fear',
 'fear',
 'anticipation',
 'fear',
 'fear',
 'anticipation',
 'fear',
 'fear',
 'fear',
 'surprise',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'fear',
 'sadness',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'fear',
 'fear',
 'anger',
 'anticipation',
 'fear',
 'fear',
 'joy',
 'anticipation',
 'fear',
 'trust',
 'joy',
 'trust',
 'fear',
 'joy',
 'fear',
 'fear',
 'anticipation',
 'sadness',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'sadness',
 'fear',
 'fear',
 'fear',
 'anticipation',
 'fear',
 'sadness',
 'sadness',
 'fear',
 'fear',
 'fear',
 'fear',
 'anticipation',
 'fear',
 'trust',
 'fear',
 'joy',
 'disgust',
 'trust',
 'sadness',
 'joy',
 'anticipation',
 'joy',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'fear',
 'sadness',
 'anticipa

In [ ]:
# Apply the classification to each row in the dataframe
df['label'] = df['stanzas'].apply(lambda x: classify_emotion(x))